In [3]:
import tensorflow as tf
print(tf.__version__)


2.15.0


# **Modified VGG16**

In [4]:

import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = "/kaggle/input/neuralnetweorkdata/dataset/train"
test_dir = "/kaggle/input/neuralnetweorkdata/dataset/test"

# Set the target image size
img_width, img_height = 224, 224

# Set the batch size
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 20% of the data will be used for validation
)

# Create the training data generator
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'  # Specify that this is the training subset
)

# Create the validation data generator
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # Specify that this is the validation subset
)

model = Sequential()

model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(img_width, img_height, 3)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(64,(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())

model.add(Dense(4096, activation='relu'))  # 3000 = No of neurons
model.add(Dropout(0.1))  # regularization
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.1))

model.add(Dense(train_generator.num_classes, activation='softmax'))

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
epochs =20
model.fit(train_generator, epochs=epochs, validation_data=validation_generator)

# Calculate accuracy on the validation set
val_loss, val_accuracy = model.evaluate(validation_generator)
print("Validation accuracy:", val_accuracy)

test_files = [f for f in os.listdir(test_dir) if os.path.isfile(os.path.join(test_dir, f))]

test_images = []
for filename in test_files:
    img_path = os.path.join(test_dir, filename)
    img = load_img(img_path, target_size=(img_width, img_height))
    img_array = img_to_array(img) / 255.0  # Normalize pixel values to [0, 1]
    test_images.append(img_array)

# Convert the list of images to a numpy array
test_images = np.array(test_images)

# Predict the classes for the test images
predictions = model.predict(test_images)

# Convert predictions to class labels
predicted_labels = np.argmax(predictions, axis=1) + 1

# Print the predicted labels
print("Predicted labels:", predicted_labels)

Found 7920 images belonging to 5 classes.
Found 1980 images belonging to 5 classes.


/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/20


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1710984597.971636     255 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1710984597.992626     255 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


248/248 ━━━━━━━━━━━━━━━━━━━━ 0s 728ms/step - accuracy: 0.2059 - loss: 1.6005

W0000 00:00:1710984780.320142     254 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


248/248 ━━━━━━━━━━━━━━━━━━━━ 299s 935ms/step - accuracy: 0.2060 - loss: 1.6004 - val_accuracy: 0.3561 - val_loss: 1.4011
Epoch 2/20
248/248 ━━━━━━━━━━━━━━━━━━━━ 176s 693ms/step - accuracy: 0.4087 - loss: 1.2720 - val_accuracy: 0.4773 - val_loss: 1.1892
Epoch 3/20
248/248 ━━━━━━━━━━━━━━━━━━━━ 175s 691ms/step - accuracy: 0.5314 - loss: 1.1058 - val_accuracy: 0.4702 - val_loss: 1.2383
Epoch 4/20
248/248 ━━━━━━━━━━━━━━━━━━━━ 175s 691ms/step - accuracy: 0.5590 - loss: 1.0300 - val_accuracy: 0.5409 - val_loss: 1.1191
Epoch 5/20
248/248 ━━━━━━━━━━━━━━━━━━━━ 175s 690ms/step - accuracy: 0.5990 - loss: 0.9741 - val_accuracy: 0.5737 - val_loss: 1.1025
Epoch 6/20
248/248 ━━━━━━━━━━━━━━━━━━━━ 174s 688ms/step - accuracy: 0.6156 - loss: 0.9293 - val_accuracy: 0.5697 - val_loss: 1.0652
Epoch 7/20
248/248 ━━━━━━━━━━━━━━━━━━━━ 173s 686ms/step - accuracy: 0.6285 - loss: 0.9303 - val_accuracy: 0.5379 - val_loss: 1.1287
Epoch 8/20
248/248 ━━━━━━━━━━━━━━━━━━━━ 176s 695ms/step - accuracy: 0.6466 - loss: 0.88

W0000 00:00:1710988169.817867     256 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


4/4 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step  
Predicted labels: [2 4 5 2 2 2 5 1 4 1 3 3 1 5 2 3 4 3 3 1 4 5 5 4 2 1 1 1 2 4 5 1 1 3 4 3 1
 1 4 4 1 5 2 3 2 1 2 5 4 4 4 4 5 2 4 2 5 3 5 5 4 2 5 5 2 3 2 4 5 5 3 3 5 1
 4 1 2 2 5 2 5 1 2 5 5 2 4 4 1 3 1 3 4 4 1 2 1 2 3 4]


# CNN MODEL

In [5]:

import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = "/kaggle/input/neuralnetweorkdata/dataset/train"
test_dir = "/kaggle/input/neuralnetweorkdata/dataset/test"

# Set the target image size
img_width, img_height = 299, 299

# Set the batch size
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 20% of the data will be used for validation
)

# Create the training data generator
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'  # Specify that this is the training subset
)

# Create the validation data generator
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # Specify that this is the validation subset
)

model = Sequential()

model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(img_width, img_height, 3)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())

model.add(Dense(3000, activation='relu'))  # 3000 = No of neurons
model.add(Dropout(0.1))  # regularization
model.add(Dense(3000, activation='relu'))
model.add(Dropout(0.1))

model.add(Dense(train_generator.num_classes, activation='softmax'))

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
epochs =20

model.fit(train_generator, epochs=epochs, validation_data=validation_generator)

# Calculate accuracy on the validation set
val_loss, val_accuracy = model.evaluate(validation_generator)
print("Validation accuracy:", val_accuracy)

test_files = [f for f in os.listdir(test_dir) if os.path.isfile(os.path.join(test_dir, f))]

test_images = []
for filename in test_files:
    img_path = os.path.join(test_dir, filename)
    img = load_img(img_path, target_size=(img_width, img_height))
    img_array = img_to_array(img) / 255.0  # Normalize pixel values to [0, 1]
    test_images.append(img_array)

# Convert the list of images to a numpy array
test_images = np.array(test_images)

# Predict the classes for the test images
predictions = model.predict(test_images)

# Convert predictions to class labels
predicted_labels = np.argmax(predictions, axis=1) + 1

# Print the predicted labels
print("Predicted labels:", predicted_labels)

Found 7920 images belonging to 5 classes.
Found 1980 images belonging to 5 classes.


/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/20


W0000 00:00:1710988243.010954     255 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


 57/248 ━━━━━━━━━━━━━━━━━━━━ 4:17 1s/step - accuracy: 0.2519 - loss: 1.5583   

...

W0000 00:00:1710988522.397184     256 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


248/248 ━━━━━━━━━━━━━━━━━━━━ 235s 926ms/step - accuracy: 0.5834 - loss: 1.0175 - val_accuracy: 0.5056 - val_loss: 1.2141
Epoch 3/20
248/248 ━━━━━━━━━━━━━━━━━━━━ 233s 919ms/step - accuracy: 0.6397 - loss: 0.9031 - val_accuracy: 0.5702 - val_loss: 1.0399
Epoch 4/20
248/248 ━━━━━━━━━━━━━━━━━━━━ 233s 919ms/step - accuracy: 0.6753 - loss: 0.8254 - val_accuracy: 0.5955 - val_loss: 1.0000
Epoch 5/20
248/248 ━━━━━━━━━━━━━━━━━━━━ 233s 918ms/step - accuracy: 0.7055 - loss: 0.7559 - val_accuracy: 0.6460 - val_loss: 0.8979
Epoch 6/20
248/248 ━━━━━━━━━━━━━━━━━━━━ 233s 917ms/step - accuracy: 0.7342 - loss: 0.6845 - val_accuracy: 0.6606 - val_loss: 0.9157
Epoch 7/20
248/248 ━━━━━━━━━━━━━━━━━━━━ 235s 925ms/step - accuracy: 0.7534 - loss: 0.6526 - val_accuracy: 0.6778 - val_loss: 0.8668
Epoch 8/20
248/248 ━━━━━━━━━━━━━━━━━━━━ 233s 919ms/step - accuracy: 0.7664 - loss: 0.6007 - val_accuracy: 0.6692 - val_loss: 0.8871
Epoch 9/20
248/248 ━━━━━━━━━━━━━━━━━━━━ 235s 925ms/step - accuracy: 0.7998 - loss: 0.53

W0000 00:00:1710993011.714334     257 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


4/4 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step 
Predicted labels: [3 4 5 2 2 2 1 1 4 1 3 3 1 5 2 3 4 4 3 5 4 5 5 2 2 1 1 1 5 2 5 1 1 3 4 3 5
 1 4 4 5 5 2 3 2 3 2 5 4 4 2 3 5 4 4 4 5 3 1 5 3 3 5 1 2 3 2 3 5 1 3 3 5 1
 2 1 2 2 5 2 5 5 2 5 5 2 4 4 3 3 3 3 2 4 1 2 1 2 3 3]


W0000 00:00:1710993016.861045     254 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


In [6]:
pip install einops


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 570.8 kB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


# Transformers

In [7]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision.models import resnet18
from torch.nn import functional as F
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torchvision.datasets import ImageFolder
from torch.utils.data import random_split
from torchvision import transforms
from einops import rearrange
import math 
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import sys

# Set the target image size
img_width, img_height = 299, 299

# Set the batch size
batch_size = 50

# Set the paths for the train and test directories
train_dir = "/kaggle/input/neuralnetweorkdata/dataset/train"
test_dir = "/kaggle/input/neuralnetweorkdata/dataset/test"
# Set up image data generators with data augmentation for the training set
train_transform = transforms.Compose([
    transforms.RandomResizedCrop((img_height, img_width)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

# Create the training data loader with the defined transformations
train_dataset = ImageFolder(train_dir, transform=train_transform)

# Split the dataset into training and validation sets
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

# Create data loaders for training and validation sets
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

class PatchEmbedding(nn.Module):
    def __init__(self, img_size, patch_size, in_channels, embed_dim):
        super(PatchEmbedding, self).__init__()
        self.img_size = img_size
        self.patch_size = patch_size
        self.in_channels = in_channels
        self.embed_dim = embed_dim

    def forward(self, x):
        x = rearrange(x, 'b c h w -> b (h w c)')
        #print("Shape after rearrangement:", x.shape)
        
        # Initialize the linear layer based on the shape of x
        self.projection = nn.Linear(x.shape[1], self.embed_dim)
        x = self.projection(x)
        return x


class PositionalEncoding(nn.Module):
    def __init__(self, embed_dim, max_seq_len=512):
        super(PositionalEncoding, self).__init__()
        self.embed_dim = embed_dim
        self.register_buffer('pe', self._get_positional_encoding(max_seq_len, embed_dim))

    def _get_positional_encoding(self, max_seq_len, embed_dim):
        position = torch.arange(0, max_seq_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, embed_dim, 2).float() * -(math.log(10000.0) / embed_dim))
        pe = torch.zeros(max_seq_len, embed_dim)
        pe[:, 0::2] = torch.sin(position * div_term) #even positions
        pe[:, 1::2] = torch.cos(position * div_term)  #odd positions
        pe = pe.unsqueeze(0)
        return pe

    def forward(self, x):
        # Check if x is 3D (batch_size, seq_len, embed_dim)
        if x.dim() == 3:
            return x + self.pe[:, :x.size(1)].detach()
        else:
            raise ValueError(f"Unsupported input shape: {x.shape}")





# Transformer Encoder block

class TransformerEncoderBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, mlp_dim):
        super(TransformerEncoderBlock, self).__init__()
        self.embed_dim = embed_dim
        self.ln1 = nn.LayerNorm(embed_dim)
        self.attention = nn.MultiheadAttention(embed_dim, num_heads) #num_head==> more than head (hyperparam)
        self.ln2 = nn.LayerNorm(embed_dim)
        self.mlp = MLPBlock(embed_dim, mlp_dim)

    def forward(self, x):
        # Self-attention
        residual = x
        x = self.ln1(x)
        x = self.attention(x, x, x)[0]
        x = x + residual

        # MLP
        residual = x
        x = self.ln2(x)
        x = self.mlp(x)
        x = x + residual

        return x
    
    
# MLP block
class MLPBlock(nn.Sequential):  # training here of features
    def __init__(self, embed_dim, mlp_dim):
        super(MLPBlock, self).__init__(
            nn.Linear(embed_dim, mlp_dim), #layer
            nn.GELU(), #act fn 
            nn.Dropout(0.1), #no overfitting
            nn.Linear(mlp_dim, embed_dim), #layer
            nn.Dropout(0.1)
        )
        
        
class VisionTransformer(nn.Module):
    def __init__(self, img_size, patch_size, in_channels, num_classes, embed_dim, num_heads, mlp_dim, num_layers):
        super(VisionTransformer, self).__init__()
        
        self.patch_embedding = PatchEmbedding(img_size, patch_size, in_channels, embed_dim)
        self.positional_encoding = PositionalEncoding(embed_dim)
        self.transformer_blocks = nn.ModuleList([
            TransformerEncoderBlock(embed_dim, num_heads, mlp_dim)
            for _ in range(num_layers)
        ])

        self.ln = nn.LayerNorm(embed_dim)
        self.fc = nn.Linear(embed_dim, num_classes)

    def forward(self, x):
        x = self.patch_embedding(x)
        x = x.view(x.size(0), -1, self.patch_embedding.embed_dim)  # Reshape to (batch_size, seq_len, embed_dim)
        x = self.positional_encoding(x)
        
        for block in self.transformer_blocks:
            x = block(x)

        x = self.ln(x.mean(dim=1))  # Global average pooling
        x = self.fc(x)

        return x

# Initialize the model
model = VisionTransformer(
    img_size=(img_width, img_height),
    patch_size=16,
    in_channels=3,
    num_classes=5,
    embed_dim=256, #size of vector that out from embbeding #before = 256
    num_heads=8, #before = 8 
    mlp_dim=512,  #before=512
    num_layers=6 #before=6
)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters()) #upadte weights

import sys

epochs =1
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    # Print the length of the training generator
    print(f'Training generator length: {len(train_loader)}')

    # Use the data generated by train_generator
    for i, (inputs, labels) in enumerate(train_loader):
        # Convert inputs and labels to PyTorch tensors
        inputs, labels = torch.tensor(inputs), torch.tensor(labels)

        # Ensure the inputs are in the correct format (NCHW)
        inputs = inputs.permute(0, 3, 1, 2).contiguous()


        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        # Calculate training accuracy
        _, predicted = torch.max(outputs, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels.argmax(dim=0)).sum().item()

        # Print loss for each iteration
        print(f'Epoch {epoch + 1}/{epochs}, Iteration {i + 1}/{len(train_loader)}, Loss: {loss.item():.4f}')

        # Break the loop if you want to iterate only up to len(train_generator) batches
        if i + 1 == len(train_loader):
            break

    # Calculate training accuracy
    train_accuracy = (correct_train / total_train) * 100
        # Validation phase
    model.eval()
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for inputs_val, labels_val in val_loader:
            # Convert inputs and labels to PyTorch tensors
            inputs_val, labels_val = torch.tensor(inputs_val), torch.tensor(labels_val)

            # Ensure the inputs are in the correct format (NCHW)
            inputs_val = inputs_val.permute(0, 3, 1, 2).contiguous()

            # Make predictions
            outputs_val = model(inputs_val)
            _, predicted_val = torch.max(outputs_val, 1)
            total_val += labels_val.size(0)
            correct_val += (predicted_val == labels_val.argmax(dim=0)).sum().item()

    # Calculate validation accuracy
    val_accuracy = (correct_val / total_val) * 100

    # Print epoch information with validation accuracy
    print(f'Epoch {epoch + 1}/{epochs}, '
          f'Training Loss: {running_loss / len(train_loader):.4f}, '
          f'Training Accuracy: {train_accuracy:.2f}%, '
          f'Validation Accuracy: {val_accuracy:.2f}%')
    sys.stdout.flush()  # Flush the output but
# Prepare the test data
test_files = [f for f in os.listdir(test_dir) if os.path.isfile(os.path.join(test_dir, f))]

test_images = []
for filename in test_files:
    img_path = os.path.join(test_dir, filename)
    img = load_img(img_path, target_size=(img_width, img_height))
    img_array = img_to_array(img) / 255.0  # Normalize pixel values to [0, 1]
    test_images.append(img_array)

# Convert the list of images to a numpy array
test_images = np.array(test_images)

# Convert numpy array to PyTorch tensor
test_images = torch.tensor(np.transpose(test_images, (0, 3, 1, 2)), dtype=torch.float32)


# Make predictions on the test set
with torch.no_grad():
    model.eval()

    predictions = model(test_images)

# Convert predictions to class labels
predicted_labels = torch.argmax(predictions, axis=1).numpy() + 1

# Print the predicted labels
print("Predicted labels:", predicted_labels)

# (Insert code for testing the model on the test dataset here)    

Training generator length: 159


/tmp/ipykernel_197/2705071380.py:189: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs, labels = torch.tensor(inputs), torch.tensor(labels)


Epoch 1/1, Iteration 1/159, Loss: 1.6976
Epoch 1/1, Iteration 2/159, Loss: 3.7139
Epoch 1/1, Iteration 3/159, Loss: 3.0375
Epoch 1/1, Iteration 4/159, Loss: 2.8774
Epoch 1/1, Iteration 5/159, Loss: 1.8612
Epoch 1/1, Iteration 6/159, Loss: 1.8918
Epoch 1/1, Iteration 7/159, Loss: 1.8015
Epoch 1/1, Iteration 8/159, Loss: 1.5874
Epoch 1/1, Iteration 9/159, Loss: 1.6943
Epoch 1/1, Iteration 10/159, Loss: 1.6908
Epoch 1/1, Iteration 11/159, Loss: 1.7590
Epoch 1/1, Iteration 12/159, Loss: 1.7151
Epoch 1/1, Iteration 13/159, Loss: 1.7087
Epoch 1/1, Iteration 14/159, Loss: 1.6564
Epoch 1/1, Iteration 15/159, Loss: 1.6628
Epoch 1/1, Iteration 16/159, Loss: 1.6400
Epoch 1/1, Iteration 17/159, Loss: 1.6800
Epoch 1/1, Iteration 18/159, Loss: 1.6084
Epoch 1/1, Iteration 19/159, Loss: 1.6886
Epoch 1/1, Iteration 20/159, Loss: 1.6559
Epoch 1/1, Iteration 21/159, Loss: 1.7105
Epoch 1/1, Iteration 22/159, Loss: 1.6070
Epoch 1/1, Iteration 23/159, Loss: 1.6408
Epoch 1/1, Iteration 24/159, Loss: 1.6672
E

/tmp/ipykernel_197/2705071380.py:223: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs_val, labels_val = torch.tensor(inputs_val), torch.tensor(labels_val)


Epoch 1/1, Training Loss: 1.6736, Training Accuracy: 10.35%, Validation Accuracy: 10.10%
Predicted labels: [5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 4 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 4 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 4 4 5 5 5 5 4 5 5]


# Saving Result for Submission on kaggle

In [8]:
import numpy as np
import pandas as pd
result_df = pd.DataFrame({
    'image_id': [filename.split('.')[0] for filename in test_files],  # Extracting image names without extension
    'label': predicted_labels
})

# Save the DataFrame to a CSV file
result_csv_path = "/kaggle/working/comp5.csv"
result_df.to_csv(result_csv_path, index=False)

# Print the DataFrame
print(result_df)

# Print a message indicating the CSV file path
print(f"Predictions saved to: {result_csv_path}")

   image_id  label
0      1266      5
1      1862      5
2       311      5
3      3362      5
4      3652      5
..      ...    ...
95     3813      5
96      639      5
97     3449      4
98     1656      5
99     1910      5

[100 rows x 2 columns]
Predictions saved to: /kaggle/working/comp5.csv
